# Linear Regression

In [ ]:
from plotly.graph_objects import Figure, Scatter, Image, Heatmap
from plotly.subplots      import make_subplots
from numpy                import array, linspace, arange, eye, zeros
from numpy.random         import random, randn, seed
from numpy.linalg         import inv, solve
from scipy.stats          import multivariate_normal
rand = random

## Model specification

In [ ]:
# define the features for polynomials
# `x` must be a vector
def phi(x, d):
    return x.reshape(x.shape[0],1) ** arange(d).reshape(1,d)
phi(randn(5), 3)       # test

In [ ]:
def f(x, w):
    d = w.shape[0]
    return phi(x, d) @ w
f(randn(5), randn(3))

In [ ]:
def sample(w_true, n, noise_std=0.05):
    x = rand(n)                   # uniformly distributed locations
    y = f(x, w_true) + noise_std*randn(n)  # noisy measurements
    return x, y
sample(array([0.1, 0.2]), 5)

In [ ]:
def linear_regression(x, y, d):
    X = phi(x, d)
    # maximum likelihood estimator
    #w_ml = inv(X.T @ X) @ X.T @ y
    w_ml = solve(X.T @ X, X.T @ y) # same as previous line
    return w_ml

## Toy example

In [ ]:
# create some toy data
seed(0)          # set the seed for numpy
n_train = 20
n_test  = 50
sigma   = 0.2
w_true  = array([0.1, -2.5, 3.0])
print(f"w_true = {w_true}")
x_train, y_train = sample(w_true, n_train, sigma)
x_pred  = linspace(0.0, 1.0, n_test)
# show the results for different `d`
d_candidates = [2, 3, 4, 5, 6, 7, 8, 9, 10]
fig = Figure()
fig.add_trace(Scatter(x=x_train, y=y_train, mode='markers', name='train'))
fig.add_trace(Scatter(x=x_pred, y=f(x_pred, w_true), mode='lines', name='true', line=dict(dash='dot')))
for d in d_candidates:
    w_ml    = linear_regression(x_train, y_train, d)
    y_pred  = f(x_pred, w_ml)
    fig.add_trace(Scatter(x=x_pred,  y=y_pred,  mode='lines', name=f'd={d}'))
    fig.update_layout(title='Linear regression', xaxis_title='x', yaxis_title='y')
fig.show()

## Anscombe's quartet
from https://en.wikipedia.org/wiki/Anscombe%27s_quartet

In [ ]:
x_all = array([[10.00,  8.00, 13.00,  9.00, 11.00, 14.00,  6.00,  4.00, 12.00,  7.00,  5.00],
[10.00,  8.00, 13.00,  9.00, 11.00, 14.00,  6.00,  4.00, 12.00,  7.00,  5.00],
[10.00,  8.00, 13.00,  9.00, 11.00, 14.00,  6.00,  4.00, 12.00,  7.00,  5.00],
[ 8.00,  8.00,  8.00,  8.00,  8.00,  8.00,  8.00, 19.00,  8.00,  8.00,  8.00]])
y_all = array([[ 8.04,  6.95,  7.58,  8.81,  8.33,  9.96,  7.24,  4.26, 10.84,  4.82,  5.68],
[ 9.14,  8.14,  8.74,  8.77,  9.26,  8.10,  6.13,  3.10,  9.13,  7.26,  4.74],
[ 7.46,  6.77, 12.74,  7.11,  7.81,  8.84,  6.08,  5.39,  8.15,  6.42,  5.73],
[ 6.58,  5.76,  7.71,  8.84,  8.47,  7.04,  5.25, 12.50,  5.56,  7.91,  6.89]])

In [ ]:
x_pred = linspace(3, 15, 5)
d = 2     # fit linear models
fig = make_subplots(rows=2,cols=2)
row = [1,1,2,2]
col = [1,2,1,2]
for set in range(4):
    x_set = x_all[set]
    y_set = y_all[set]
    fig.add_trace(Scatter(x=x_set, y=y_set, mode='markers', 
                          name=f'set{set+1}'), 
                  row=row[set], col=col[set])
    w_set = linear_regression(x_set, y_set, d)
    fig.add_trace(Scatter(x=x_pred, y=f(x_pred, w_set), mode='lines', 
                          name=f'pred{set+1}'),
                  row=row[set], col=col[set])
    print(w_set)
# show all results
fig.show()

# Bayesian linear regression
Reimplementing Murphy's book Figure 7.11

In [ ]:
# create some toy data
seed(0)          # set the seed for numpy
d = 2
n_train = 20
n_test  = 50
sigma   = 0.2
Sigma   = sigma**2 * eye(n_train)   # measurement noise
w_true  = 2.0 * randn(d)            # true weights
x_train, y_train = sample(w_true, n_train, sigma)
x_pred  = linspace(0.0, 1.0, n_test)

In [ ]:
# prior for `w`
d  = 2
w0 = zeros(d)
V0 = eye(d)

In [ ]:
# posterior for `w` after seeing data points `x`, `y`
# `x` and `y` must have shape (n,)
def posterior_w(x, y, w0, V0, Sigma):
    d = w0.shape[0]
    X = phi(x, d)
    invV0    = inv(V0)
    invSigma = inv(Sigma)
    Vn = inv(X.T @ invSigma @ X + invV0)
    wn = Vn @ (invV0 @ w0 + X.T @ invSigma @ y)
    return wn, Vn
posterior_w(x_train, y_train, w0, V0, Sigma)

In [ ]:
def sample_w(w0, V0):
    d = w0.shape[0]
    # `cholesky(V0)` is a matrix squareroot of `V0`
    return cholesky(V0) @ randn(d) + w0

In [ ]:
def gaussian_pdf_image(mu, Sigma, x0_range, x1_range):
    assert mu.shape[0]==2
    n = 100
    pdf = lambda x0, x1: multivariate_normal.pdf(array([x0,x1]), mu, Sigma)
    return array([[pdf(x0,x1) for x1 in x1_range] for x0 in x0_range])

In [ ]:
n_sel = [0, 1, 2, 20]
fig = make_subplots(rows=len(n_sel), cols=3)
row = 1
for n in n_sel:
    wi, Vi = posterior_w(x_train[:n], y_train[:n], w0, V0, Sigma[:n,:n])
    x0_range = linspace(-5.0, 5.0)
    x1_range = linspace(-5.0, 5.0)
    fig.add_trace(Heatmap(x=x0_range, y=x1_range, 
                          z=gaussian_pdf_image(wi, Vi, x0_range, x1_range), 
                          zmin=0.0, zmax=0.15,
                          showscale=False), row=row, col=2)
    fig.update_yaxes(row=row, col=2, scaleanchor=fig.data[-1]['xaxis'])
    row = row + 1
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.show()